In [45]:
##dados finais - preparacao
!pip install -q unidecode

import pandas as pd
import re
from unidecode import unidecode

df_base = pd.read_csv('/content/BASE_COM_DADOS_CENSITARIOS_pos_join.csv')
nas = pd.read_csv('/content/anuncios_codigos_faltantes.csv')


df_base_expand = pd.concat([df_base, nas], ignore_index=True)

# Calcular a média da coluna 'RENDA BRUTA ANUAL' por distrito, ignorando valores nulos
media_renda_por_distrito = df_base_expand.groupby('DISTRITO')['RENDA BRUTA ANUAL '].transform('mean')

# Substituir os valores nulos da coluna 'RENDA BRUTA ANUAL' por a média calculada
df_base_expand['RENDA BRUTA ANUAL '] = df_base_expand['RENDA BRUTA ANUAL '].fillna(media_renda_por_distrito)
df_base_expand = df_base_expand.dropna(subset=['RENDA BRUTA ANUAL '])

df_base.shape

(1645, 37)

In [47]:
#DF AGRUPADO POR DISTRITO
df_grouped = df_base_expand.groupby('DISTRITO').agg(
    NUM_DE_ANUNCIOS=('anuncio/propriedade', 'count'),
    VALOR_MEDIANA_arrendamento_MENSAL=('valor renda(€/mes)', 'median'),
    MEDIANA_AREA_CASA=('area(m²)', 'median'),
    MEDIA_RENDA_bruta_anual_INDIVIDUO=('RENDA BRUTA ANUAL ', 'mean')

)

df_grouped.head(20)


,NUM_DE_ANUNCIOS,VALOR_MEDIANA_arrendamento_MENSAL,MEDIANA_AREA_CASA,MEDIA_RENDA_bruta_anual_INDIVIDUO
DISTRITO,,,,
AVEIRO,55,920.0,91.3,9114.075000
BRAGA,69,1100.0,126.0,8497.133333
BRAGANCA,4,365.0,55.0,6683.000000
CASTELO BRANCO,4,450.0,52.7,7492.000000
COIMBRA,55,750.0,81.0,10068.622222
EVORA,3,990.0,94.0,8019.000000
FARO,49,2500.0,136.5,8216.500000
LEIRIA,37,1100.0,125.0,8286.838710
LISBOA,981,2700.0,120.0,12056.968205


In [5]:
## CONSUMO DA GEOAPI
import requests

distritos = ["aveiro", "beja", "braga", "braganca", "castelo branco", "coimbra", "evora", "faro", "guarda", "leiria", "lisboa", "portalegre", "porto", "santarem", "setubal", "viana do castelo", "vila real", "viseu"]

# URL base da API GeoAPI.pt
base_url = "https://json.geoapi.pt/distrito/"

# FunC para obter dados de um distrito
def obter_dados_distrito(distrito):
    response = requests.get(base_url + distrito)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Falha ao obter dados para {distrito}: {response.status_code}")
        return None

#armazenar os dados dos distritos
dados_lista = []

#iterar sobre cada distrito e coletar os dados
for distrito in distritos:
    dados = obter_dados_distrito(distrito)
    if dados and 'censos2021' in dados:
        censos = dados['censos2021']
        censos['distrito'] = distrito
        dados_lista.append(censos)


df = pd.DataFrame(dados_lista)
print(df)


    N_EDIFICIOS_CLASSICOS  N_EDIFICIOS_CLASS_CONST_1_OU_2_ALOJ  \
0                  257230                               242350   
1                   94143                                91630   
2                  276686                               261081   
3                   84576                                81978   
4                  116715                               111502   
5                  188245                               177322   
6                   80507                                77139   
7                  201078                               180012   
8                  114698                               111860   
9                  215874                               204213   
10                 365844                               273053   
11                  67444                                65208   
12                 443651                               392607   
13                 209481                               200389   
14        

In [48]:
# Colocar a coluna 'distrito' como a primeira coluna do DataFrame
colunas = ['distrito'] + [col for col in df.columns if col != 'distrito']
df_api = df[colunas]
# Lista de colunas p manter
colunas_para_manter = [
    'distrito', 'N_EDIFICIOS_CLASSICOS', 'N_EDIFICIOS_EXCLUSIV_RESID',
    'N_EDIFICIOS_1_OU_2_PISOS', 'N_EDIFICIOS_3_OU_MAIS_PISOS',
    'N_EDIFICIOS_COM_NECESSIDADES_REPARACAO',
    'N_ALOJAMENTOS_TOTAL', 'N_ALOJAMENTOS_FAMILIARES',
    'N_ALOJAMENTOS_FAM_CLASS_RHABITUAL', 'N_ALOJAMENTOS_FAM_CLASS_VAGOS_OU_RESID_SECUNDARIA',
    'N_RHABITUAL_COM_ESTACIONAMENTO',
    'N_RHABITUAL_PROP_OCUP', 'N_RHABITUAL_ARRENDADOS',
    'N_AGREGADOS_DOMESTICOS_PRIVADOS', 'N_NUCLEOS_FAMILIARES',
    'N_INDIVIDUOS'
]

df_api = df_api[colunas_para_manter]

df_api = df_api.rename(columns={
    'distrito': 'DISTRITO'})

df_api['DISTRITO'] = df_api['DISTRITO'].str.upper()


df_api

,DISTRITO,N_EDIFICIOS_CLASSICOS,N_EDIFICIOS_EXCLUSIV_RESID,N_EDIFICIOS_1_OU_2_PISOS,N_EDIFICIOS_3_OU_MAIS_PISOS,N_EDIFICIOS_COM_NECESSIDADES_REPARACAO,N_ALOJAMENTOS_TOTAL,N_ALOJAMENTOS_FAMILIARES,N_ALOJAMENTOS_FAM_CLASS_RHABITUAL,N_ALOJAMENTOS_FAM_CLASS_VAGOS_OU_RESID_SECUNDARIA,N_RHABITUAL_COM_ESTACIONAMENTO,N_RHABITUAL_PROP_OCUP,N_RHABITUAL_ARRENDADOS,N_AGREGADOS_DOMESTICOS_PRIVADOS,N_NUCLEOS_FAMILIARES,N_INDIVIDUOS
0,AVEIRO,257230,256525,226740,30490,107342,358103,357761,269857,87536,212506,199159,49462,270370,219491,700787
1,BEJA,94143,93968,91439,2704,26588,108137,107868,58572,48889,25889,45243,7378,59109,41460,144401
2,BRAGA,276686,275873,232011,44675,93416,399145,398743,301898,96708,236812,215247,61245,302140,255644,832373
3,BRAGANCA,84576,84417,72476,12100,31267,100472,100316,50704,49548,32140,41722,5998,50788,36529,118863
4,CASTELO BRANCO,116715,116402,93884,22831,45361,152449,152254,77679,74526,39902,59995,12826,77750,54412,177962
5,COIMBRA,188245,187581,156771,31474,82758,268161,267815,168911,98788,117771,129733,27911,169095,125363,408551
6,EVORA,80507,80185,77286,3221,20022,99107,98936,63726,35113,32072,44592,13350,63848,46316,152444
7,FARO,201078,198114,172428,28650,58878,392315,392047,193395,198021,93535,129301,44545,194192,141290,467343
8,GUARDA,114698,114575,90221,24477,38125,131553,131346,60741,70579,39285,49780,6957,60783,43407,142974
9,LEIRIA,215874,214851,192726,23148,71461,293335,293022,187235,105626,138433,144200,30338,187499,140204,458605


In [7]:
df_api.to_csv('dados_API_DISTRITOS.csv', index=False)

In [4]:
!pip install -q unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.7 MB/s eta 0:00:00


In [49]:
##dados do sef sobre estrangeiros

df_estrangeiros = pd.read_excel('/content/Mapa_6_27_2024.xlsx')
df_estrangeiros['Distrito'] = df_estrangeiros['Distrito'].str.upper()
df_estrangeiros = df_estrangeiros.rename(columns={'Distrito': 'DISTRITO'})
df_estrangeiros['DISTRITO'] = df_estrangeiros['DISTRITO'].apply(unidecode)
df_estrangeiros['DISTRITO'] = df_estrangeiros['DISTRITO'].str.strip()
df_estrangeiros

,DISTRITO,Total_ESTRANGEIROS_RESIDENTES_2021
0,AVEIRO,20395
1,BEJA,15953
2,BRAGA,23619
3,BRAGANCA,3856
4,CASTELO BRANCO,7006
5,COIMBRA,17565
6,EVORA,4903
7,FARO,105142
8,GUARDA,2685
9,LEIRIA,26952


In [50]:
df_api_sef = pd.merge(df_api, df_estrangeiros, on='DISTRITO', how='inner')
df_api_sef

,DISTRITO,N_EDIFICIOS_CLASSICOS,N_EDIFICIOS_EXCLUSIV_RESID,N_EDIFICIOS_1_OU_2_PISOS,N_EDIFICIOS_3_OU_MAIS_PISOS,N_EDIFICIOS_COM_NECESSIDADES_REPARACAO,N_ALOJAMENTOS_TOTAL,N_ALOJAMENTOS_FAMILIARES,N_ALOJAMENTOS_FAM_CLASS_RHABITUAL,N_ALOJAMENTOS_FAM_CLASS_VAGOS_OU_RESID_SECUNDARIA,N_RHABITUAL_COM_ESTACIONAMENTO,N_RHABITUAL_PROP_OCUP,N_RHABITUAL_ARRENDADOS,N_AGREGADOS_DOMESTICOS_PRIVADOS,N_NUCLEOS_FAMILIARES,N_INDIVIDUOS,Total_ESTRANGEIROS_RESIDENTES_2021
0,AVEIRO,257230,256525,226740,30490,107342,358103,357761,269857,87536,212506,199159,49462,270370,219491,700787,20395
1,BEJA,94143,93968,91439,2704,26588,108137,107868,58572,48889,25889,45243,7378,59109,41460,144401,15953
2,BRAGA,276686,275873,232011,44675,93416,399145,398743,301898,96708,236812,215247,61245,302140,255644,832373,23619
3,BRAGANCA,84576,84417,72476,12100,31267,100472,100316,50704,49548,32140,41722,5998,50788,36529,118863,3856
4,CASTELO BRANCO,116715,116402,93884,22831,45361,152449,152254,77679,74526,39902,59995,12826,77750,54412,177962,7006
5,COIMBRA,188245,187581,156771,31474,82758,268161,267815,168911,98788,117771,129733,27911,169095,125363,408551,17565
6,EVORA,80507,80185,77286,3221,20022,99107,98936,63726,35113,32072,44592,13350,63848,46316,152444,4903
7,FARO,201078,198114,172428,28650,58878,392315,392047,193395,198021,93535,129301,44545,194192,141290,467343,105142
8,GUARDA,114698,114575,90221,24477,38125,131553,131346,60741,70579,39285,49780,6957,60783,43407,142974,2685
9,LEIRIA,215874,214851,192726,23148,71461,293335,293022,187235,105626,138433,144200,30338,187499,140204,458605,26952


## Último join/tratamento para elaboração do DF FINAL P/ ArcGis

In [51]:
df_final = pd.merge(df_grouped, df_api_sef, on='DISTRITO', how='inner')
df_final

,DISTRITO,NUM_DE_ANUNCIOS,VALOR_MEDIANA_arrendamento_MENSAL,MEDIANA_AREA_CASA,MEDIA_RENDA_bruta_anual_INDIVIDUO,N_EDIFICIOS_CLASSICOS,N_EDIFICIOS_EXCLUSIV_RESID,N_EDIFICIOS_1_OU_2_PISOS,N_EDIFICIOS_3_OU_MAIS_PISOS,N_EDIFICIOS_COM_NECESSIDADES_REPARACAO,...,N_ALOJAMENTOS_FAMILIARES,N_ALOJAMENTOS_FAM_CLASS_RHABITUAL,N_ALOJAMENTOS_FAM_CLASS_VAGOS_OU_RESID_SECUNDARIA,N_RHABITUAL_COM_ESTACIONAMENTO,N_RHABITUAL_PROP_OCUP,N_RHABITUAL_ARRENDADOS,N_AGREGADOS_DOMESTICOS_PRIVADOS,N_NUCLEOS_FAMILIARES,N_INDIVIDUOS,Total_ESTRANGEIROS_RESIDENTES_2021
0,AVEIRO,55,920.0,91.3,9114.075000,257230,256525,226740,30490,107342,...,357761,269857,87536,212506,199159,49462,270370,219491,700787,20395
1,BRAGA,69,1100.0,126.0,8497.133333,276686,275873,232011,44675,93416,...,398743,301898,96708,236812,215247,61245,302140,255644,832373,23619
2,BRAGANCA,4,365.0,55.0,6683.000000,84576,84417,72476,12100,31267,...,100316,50704,49548,32140,41722,5998,50788,36529,118863,3856
3,CASTELO BRANCO,4,450.0,52.7,7492.000000,116715,116402,93884,22831,45361,...,152254,77679,74526,39902,59995,12826,77750,54412,177962,7006
4,COIMBRA,55,750.0,81.0,10068.622222,188245,187581,156771,31474,82758,...,267815,168911,98788,117771,129733,27911,169095,125363,408551,17565
5,EVORA,3,990.0,94.0,8019.000000,80507,80185,77286,3221,20022,...,98936,63726,35113,32072,44592,13350,63848,46316,152444,4903
6,FARO,49,2500.0,136.5,8216.500000,201078,198114,172428,28650,58878,...,392047,193395,198021,93535,129301,44545,194192,141290,467343,105142
7,LEIRIA,37,1100.0,125.0,8286.838710,215874,214851,192726,23148,71461,...,293022,187235,105626,138433,144200,30338,187499,140204,458605,26952
8,LISBOA,981,2700.0,120.0,12056.968205,365844,355301,248767,117077,124932,...,1189145,942350,246315,355472,590290,283718,943439,660847,2275385,294736
9,PORTO,359,1400.0,105.0,10230.465465,443651,439844,351220,92431,175248,...,859207,692854,166027,482744,449248,186930,693399,550474,1784277,55473


In [52]:
df_final.columns

Index(['DISTRITO', 'NUM_DE_ANUNCIOS', 'VALOR_MEDIANA_arrendamento_MENSAL',
       'MEDIANA_AREA_CASA', 'MEDIA_RENDA_bruta_anual_INDIVIDUO',
       'N_EDIFICIOS_CLASSICOS', 'N_EDIFICIOS_EXCLUSIV_RESID',
       'N_EDIFICIOS_1_OU_2_PISOS', 'N_EDIFICIOS_3_OU_MAIS_PISOS',
       'N_EDIFICIOS_COM_NECESSIDADES_REPARACAO', 'N_ALOJAMENTOS_TOTAL',
       'N_ALOJAMENTOS_FAMILIARES', 'N_ALOJAMENTOS_FAM_CLASS_RHABITUAL',
       'N_ALOJAMENTOS_FAM_CLASS_VAGOS_OU_RESID_SECUNDARIA',
       'N_RHABITUAL_COM_ESTACIONAMENTO', 'N_RHABITUAL_PROP_OCUP',
       'N_RHABITUAL_ARRENDADOS', 'N_AGREGADOS_DOMESTICOS_PRIVADOS',
       'N_NUCLEOS_FAMILIARES', 'N_INDIVIDUOS',
       'Total_ESTRANGEIROS_RESIDENTES_2021'],
      dtype='object')

In [53]:
df_final.to_csv('df_final.csv', index=False)

In [19]:
##Planilha de renda corrigida pela inflacao
df = pd.read_csv('/content/df_final.csv')
df['RENDA_MEDIA_BRUTA_ANUAL_corrigida_inflacao'] = df['MEDIA_RENDA_bruta_anual_INDIVIDUO'] * 1.1775
df['RENDA_BRUTA_MENSAL'] = df['RENDA_MEDIA_BRUTA_ANUAL_corrigida_inflacao'] / 12


In [20]:
##Se renda cobre o aluguer: 1, else: 0. Base para mapa arcgis online
def renda_suficiente(row):
  if row['RENDA_BRUTA_MENSAL'] >= row['VALOR_MEDIANA_arrendamento_MENSAL']:
    return 1
  else:
    return 0

df['RENDA_SUFICIENTE'] = df.apply(renda_suficiente, axis=1)

In [21]:
df_renda_suficiente = df[['DISTRITO', 'VALOR_MEDIANA_arrendamento_MENSAL', 'RENDA_MEDIA_BRUTA_ANUAL_corrigida_inflacao',
                          'RENDA_BRUTA_MENSAL', 'RENDA_SUFICIENTE']].copy()

In [22]:
df_renda_suficiente.head()

,DISTRITO,VALOR_MEDIANA_arrendamento_MENSAL,RENDA_MEDIA_BRUTA_ANUAL_corrigida_inflacao,RENDA_BRUTA_MENSAL,RENDA_SUFICIENTE
0,AVEIRO,920.0,10731.823313,894.318609,0
1,BRAGA,1100.0,10005.374500,833.781208,0
2,BRAGANCA,365.0,7869.232500,655.769375,1
3,CASTELO BRANCO,450.0,8821.830000,735.152500,1
4,COIMBRA,750.0,11855.802667,987.983556,1


In [23]:
df_renda_suficiente.to_csv('df_renda_suficiente.csv', index=False)

## Dados do SEF sobre imigracao 2021

In [5]:
##retrabalho nos dados do SEF. dessa vez para add dados de 2020 e variacoes

# montagem do df
data = {
    "Distritos": ["Lisboa", "Faro", "Setúbal", "Porto", "Leiria", "Braga", "Aveiro", "Coimbra", "Santarém", "Beja",
                  "Madeira", "Castelo Branco", "Viseu", "Viana do Castelo", "Évora", "Açores", "Bragança",
                  "Portalegre", "Vila Real", "Guarda"],
    "Número de Estrangeiros residentes em 2020": [285570, 103565, 60939, 50238, 24788, 21113, 18517, 17028,
                                                  16378, 14095, 9455, 6717, 6503, 6167, 4802, 4090, 4036,
                                                  2756, 2731, 2607],
    "% face ao total de estrangeiros residentes 2020": [43.1, 15.6, 9.2, 7.6, 3.7, 3.2, 2.8, 2.6, 2.5, 2.1,
                                                       1.4, 1.0, 1.0, 0.9, 0.7, 0.6, 0.6, 0.4, 0.4, 0.4],
    "Variação 2019-2020 (%)": [9.6, 11.8, 17.2, 18.6, 15.6, 15.8, 12.0, 9.6, 15.5, 15.8, 10.1, 14.1, 18.4, 28.8,
                               6.6, 4.9, -3.0, 8.8, 6.1, 8.7]
}

df_imig_2020 = pd.DataFrame(data)

indices_a_remover = [10, 15]
df_imig_2020 = df_imig_2020.drop(df_imig_2020.index[indices_a_remover])

df_imig_2020['Distritos'] = df_imig_2020['Distritos'].str.upper()
df_imig_2020 = df_imig_2020.rename(columns={'Distritos': 'DISTRITO'})
df_imig_2020['DISTRITO'] = df_imig_2020['DISTRITO'].apply(unidecode)
df_imig_2020['DISTRITO'] = df_imig_2020['DISTRITO'].str.strip()

df_imig_2020.head(20)


,DISTRITO,Número de Estrangeiros residentes em 2020,% face ao total de estrangeiros residentes 2020,Variação 2019-2020 (%)
0,LISBOA,285570,43.1,9.6
1,FARO,103565,15.6,11.8
2,SETUBAL,60939,9.2,17.2
3,PORTO,50238,7.6,18.6
4,LEIRIA,24788,3.7,15.6
5,BRAGA,21113,3.2,15.8
6,AVEIRO,18517,2.8,12.0
7,COIMBRA,17028,2.6,9.6
8,SANTAREM,16378,2.5,15.5
9,BEJA,14095,2.1,15.8


In [16]:
## Import dos dados do SEF de 2021 para fazer join com 2020
df_dados_2021 = pd.read_excel('/content/Mapa_6_27_2024.xlsx')
df_dados_2021['DISTRITO'] = df_dados_2021['DISTRITO'].str.upper()
df_dados_2021['DISTRITO'] = df_dados_2021['DISTRITO'].apply(unidecode)
df_dados_2021['DISTRITO'] = df_dados_2021['DISTRITO'].str.strip()

df_imig_final = pd.merge(df_imig_2020, df_dados_2021, on='DISTRITO', how='inner')

# Calc o total de estrangeiros residentes em 2021
total_estrangeiros_2021 = df_imig_final['Total_ESTRANGEIROS_RESIDENTES_2021'].sum()

# Calcular o percentual de cada distrito em relação ao total de 2021
df_imig_final['% face ao total de estrangeiros residentes 2021'] = (df_imig_final['Total_ESTRANGEIROS_RESIDENTES_2021'] / total_estrangeiros_2021) * 100

# Calc a variação de 2020 para 2021 em porcentagem
df_imig_final['Variação 2020-2021 (%)'] = ((df_imig_final['Total_ESTRANGEIROS_RESIDENTES_2021'] - df_imig_final['Número de Estrangeiros residentes em 2020']) / df['Número de Estrangeiros residentes em 2020']) * 100

df_imig_final['% face ao total de estrangeiros residentes 2021'] = df_imig_final['% face ao total de estrangeiros residentes 2021'].round(1)
df_imig_final['Variação 2020-2021 (%)'] = df_imig_final['Variação 2020-2021 (%)'].round(1)

df_imig_final = df_imig_final.rename(columns={'Total_ESTRANGEIROS_RESIDENTES_2021': 'Número de Estrangeiros residentes em 2021'})
df_imig_final.head(20)


,DISTRITO,Número de Estrangeiros residentes em 2020,% face ao total de estrangeiros residentes 2020,Variação 2019-2020 (%),Número de Estrangeiros residentes em 2021,% face ao total de estrangeiros residentes 2021,Variação 2020-2021 (%)
0,LISBOA,285570,43.1,9.6,294736,43.1,3.2
1,FARO,103565,15.6,11.8,105142,15.4,1.5
2,SETUBAL,60939,9.2,17.2,66901,9.8,9.8
3,PORTO,50238,7.6,18.6,55473,8.1,10.4
4,LEIRIA,24788,3.7,15.6,26952,3.9,8.7
5,BRAGA,21113,3.2,15.8,23619,3.5,11.9
6,AVEIRO,18517,2.8,12.0,20395,3.0,10.1
7,COIMBRA,17028,2.6,9.6,17565,2.6,3.2
8,SANTAREM,16378,2.5,15.5,18255,2.7,11.5
9,BEJA,14095,2.1,15.8,15953,2.3,13.2


In [18]:
df_imig_graf = df_imig_final.drop(columns=['% face ao total de estrangeiros residentes 2020', '% face ao total de estrangeiros residentes 2021'])
df_imig_graf.to_csv('df_imig_graf.csv', index=False)

In [17]:
df_imig_final.to_csv('df_imig_final.csv', index=False)

In [25]:
## top 10 municipios:
data_municipios = {
    "Municípios (REG LISBOA)": ["Lisboa", "Sintra", "Cascais", "Amadora", "Loures", "Odivelas", "Loulé", "Albufeira", "Porto", "Almada"],
    "Número de Estrangeiros 2020": [107238, 41155, 32939, 23458, 21077, 19946, 18892, 17177, 16405, 15471],
    "% face ao total de estrangeiros residentes em Portugal 2020": [16.2, 6.2, 5.0, 3.5, 3.2, 3.0, 2.9, 2.6, 2.5, 2.3]

}

# Criação do Df
df_municipios = pd.DataFrame(data_municipios)

indices_a_remover2 = [6, 7, 8]
df_municipios = df_municipios.drop(df_municipios.index[indices_a_remover2])
df_municipios.head(11)

df_municipios.to_csv('df_TOP_municipios_LIBOA_IMIG.csv', index=False)

In [8]:
dados_pop = pd.read_csv('/content/dados_API_DISTRITOS.csv')
dados_pop = dados_pop[['DISTRITO', 'N_INDIVIDUOS', 'N_ALOJAMENTOS_TOTAL']].copy()

pop_pais_2021 = dados_pop['N_INDIVIDUOS'].sum()

# percentual de cada distrito em relação ao total de 2021
dados_pop['% da população face ao total (CONTINENTAL)'] = (dados_pop['N_INDIVIDUOS'] / pop_pais_2021) * 100


dados_pop.head(20)

,DISTRITO,N_INDIVIDUOS,N_ALOJAMENTOS_TOTAL,% da população face ao total (CONTINENTAL)
0,AVEIRO,700787,358103,7.136110
1,BEJA,144401,108137,1.470435
2,BRAGA,832373,399145,8.476050
3,BRAGANCA,118863,100472,1.210381
4,CASTELO BRANCO,177962,152449,1.812186
5,COIMBRA,408551,268161,4.160273
6,EVORA,152444,99107,1.552336
7,FARO,467343,392315,4.758951
8,GUARDA,142974,131553,1.455903
9,LEIRIA,458605,293335,4.669972


In [12]:
##atribuicao de id a cada distrito para realizar join com shapefile no arcgis pro para criar MAPA 1 (CHOROPLET)
id_data = {
    "ID_1": [2, 4, 5, 6, 7, 8, 1, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20],
    "DISTRITO": ["AVEIRO", "BEJA", "BRAGA", "BRAGANCA", "CASTELO BRANCO", "COIMBRA", "EVORA", "FARO", "GUARDA", "LEIRIA", "LISBOA", "PORTALEGRE", "PORTO", "SANTAREM", "SETUBAL", "VIANA DO CASTELO", "VILA REAL", "VISEU"]
}

# DF
df_id = pd.DataFrame(id_data)

# JOIN
dados_pop_final = dados_pop.merge(df_id, on="DISTRITO")

#Colocar ID_1 no início
dados_pop_final = dados_pop_final[['ID_1', 'DISTRITO', 'N_INDIVIDUOS', 'N_ALOJAMENTOS_TOTAL', '% da população face ao total (CONTINENTAL)']]

In [13]:
dados_pop_final.head(20)

,ID_1,DISTRITO,N_INDIVIDUOS,N_ALOJAMENTOS_TOTAL,% da população face ao total (CONTINENTAL)
0,2,AVEIRO,700787,358103,7.136110
1,4,BEJA,144401,108137,1.470435
2,5,BRAGA,832373,399145,8.476050
3,6,BRAGANCA,118863,100472,1.210381
4,7,CASTELO BRANCO,177962,152449,1.812186
5,8,COIMBRA,408551,268161,4.160273
6,1,EVORA,152444,99107,1.552336
7,9,FARO,467343,392315,4.758951
8,10,GUARDA,142974,131553,1.455903
9,11,LEIRIA,458605,293335,4.669972


In [14]:
dados_pop_final.to_csv('dados_pop.csv', index=False)